In [9]:
from osgeo import gdal
import sys
import os
import pandas as pd

In [10]:
def get_extend(file_path):
    src = gdal.Open(file_path)
    ulx, xres, xskew, uly, yskew, yres  = src.GetGeoTransform()
    lrx = ulx + (src.RasterXSize * xres)
    lry = uly + (src.RasterYSize * yres)
    return(ulx,uly,lrx,lry)

In [11]:
ulx,uly,lrx,lry = get_extend('/media/philipp/ed7d22ba-5a3b-4d31-bf6c-6add6e106b3d/dtm/25-46.tif')

print('upper left corner: ', ulx, uly)
print('lower right corner: ', lrx, lry)

upper left corner:  173462.08030918773 389578.5985227945
lower right corner:  175027.55069879314 387762.213018567


## get info

In [12]:
src = gdal.Open('/media/philipp/ed7d22ba-5a3b-4d31-bf6c-6add6e106b3d/dtm/25-46.tif')

print("Driver: {}/{}".format(src.GetDriver().ShortName,
                            src.GetDriver().LongName))
print("Size is {} x {} x {}".format(src.RasterXSize,
                                    src.RasterYSize,
                                    src.RasterCount))
print("Projection is {}".format(src.GetProjection()))
geotransform = src.GetGeoTransform()
if geotransform:
    print("Origin = ({}, {})".format(geotransform[0], geotransform[3]))
    print("Pixel Size = ({}, {})".format(geotransform[1], geotransform[5]))

Driver: GTiff/GeoTIFF
Size is 1566 x 1817 x 1
Projection is PROJCS["Austria_Lambert",GEOGCS["GCS_BESSEL_AUT",DATUM["D_BESSEL_AUT",SPHEROID["Bessel_1841",6377397.155,299.1528128,AUTHORITY["EPSG","7004"]]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["latitude_of_origin",47.5],PARAMETER["central_meridian",13.333333333],PARAMETER["standard_parallel_1",46],PARAMETER["standard_parallel_2",49],PARAMETER["false_easting",400000],PARAMETER["false_northing",400000],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]
Origin = (173462.08030918773, 389578.5985227945)
Pixel Size = (0.9996618069, -0.9996618075)


In [13]:
st = '88-42.tif'
st1 = st.replace('.tif','')
st2 = st1.split('-')

In [14]:
st2

['88', '42']

In [12]:
cd = '/media/philipp/5e9929a4-cb93-48b8-8648-6c83a93e83ed/gis/dtm/'
directory = os.fsencode(cd)

extend = []

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".tif"): 
        #print(os.path.join(directory, filename))
        #print('#############################################################')
        #print(filename)
        ulx,uly,lrx,lry = get_extend(cd + filename)
        #print('upper left corner: ', ulx, uly)
        #print('lower right corner: ', lrx, lry)
        pos = filename.replace('.tif','').split('-')
        extend.append([filename, pos[0], pos[1], ulx, uly, lrx, lry])

In [13]:
df = pd.DataFrame(extend)
df.columns =['file', 'y', 'x','ulx','uly','lrx','lry']
df = df.sort_values('file')
df.to_csv('dtm.csv')

In [19]:
df

,Unnamed: 0,file,y,x,ulx,uly,lrx,lry
0,0,100-16.tif,100,16,360026.829121,315262.828057,362527.146703,315228.837334
1,1,100-17.tif,100,17,360026.829121,317762.145911,362527.146703,315262.828057
2,2,100-18.tif,100,18,360027.234669,320262.444506,362527.527239,317762.151937
3,3,100-19.tif,100,19,360027.428356,322762.733928,362526.709275,320262.453296
4,4,100-20.tif,100,20,360027.615981,325263.011413,362526.885346,322762.742338
...,...,...,...,...,...,...,...,...
6244,6244,99-75.tif,99,75,357527.317067,462762.206705,360027.529346,460262.994113
6245,6245,99-76.tif,99,76,357527.153018,465262.277605,360027.368623,462762.062002
6246,6246,99-77.tif,99,77,357526.980899,467762.506630,360027.206428,465262.281104
6247,6247,99-78.tif,99,78,357527.616560,470262.749518,360026.863399,467762.502980


In [15]:
df = pd.read_csv('dtm.csv', index_col=False)
df = df[['file', 'y', 'x','ulx','uly','lrx','lry']]

In [16]:
df

,file,y,x,ulx,uly,lrx,lry
0,100-16.tif,100,16,360026.829121,315262.828057,362527.146703,315228.837334
1,100-17.tif,100,17,360026.829121,317762.145911,362527.146703,315262.828057
2,100-18.tif,100,18,360027.234669,320262.444506,362527.527239,317762.151937
3,100-19.tif,100,19,360027.428356,322762.733928,362526.709275,320262.453296
4,100-20.tif,100,20,360027.615981,325263.011413,362526.885346,322762.742338
...,...,...,...,...,...,...,...
6244,99-75.tif,99,75,357527.317067,462762.206705,360027.529346,460262.994113
6245,99-76.tif,99,76,357527.153018,465262.277605,360027.368623,462762.062002
6246,99-77.tif,99,77,357526.980899,467762.506630,360027.206428,465262.281104
6247,99-78.tif,99,78,357527.616560,470262.749518,360026.863399,467762.502980


In [17]:
t_ulx,t_uly,t_lrx,t_lry = get_extend('/media/philipp/ed7d22ba-5a3b-4d31-bf6c-6add6e106b3d/test/256x256/ortho_1.tif')
print(t_ulx,t_lrx, t_uly,t_lry)

373757.9999999925 451232.9999999909 418695.99999999156 381706.9999999923


In [21]:
x = df[(t_ulx <= df['lrx']) & (t_lrx >= df['ulx']) & (t_uly >= df['uly']) & (t_lry <= df['lry'])]

In [22]:
x

,file,y,x,ulx,uly,lrx,lry
204,105-44.tif,105,44,372526.884384,385262.919836,375027.015103,382762.789118
205,105-45.tif,105,45,372526.884384,387762.050902,375027.015103,385262.919836
206,105-46.tif,105,46,372526.928634,390262.175430,375027.055698,387762.048367
207,105-47.tif,105,47,372526.944442,392762.300420,375027.070255,390262.174610
208,105-48.tif,105,48,372526.956037,395262.424163,375027.080979,392762.299222
...,...,...,...,...,...,...,...
1821,136-53.tif,136,53,450027.660244,407762.446080,452526.735143,405262.371551
1822,136-54.tif,136,54,450027.660244,410262.520608,452526.735143,407762.446080
1823,136-55.tif,136,55,450027.660244,412762.595137,452526.735143,410262.520608
1824,136-56.tif,136,56,450027.660244,415262.669665,452526.735143,412762.595137


In [20]:
x = df[(t_ulx <= df['lrx']) & (t_lrx >= df['ulx'])] #& (t_lry <= df['uly']) & (t_uly >= df['lry'])]
x

,file,y,x,ulx,uly,lrx,lry
190,105-30.tif,105,30,372527.103760,350262.692782,375027.302894,350217.707192
191,105-31.tif,105,31,372527.186747,352762.889747,375027.378512,350262.697980
192,105-32.tif,105,32,372527.265547,355262.079674,375027.450325,352762.894567
193,105-33.tif,105,33,372527.265547,357762.264455,375027.450325,355262.079674
194,105-34.tif,105,34,372527.410584,360262.438318,375027.582534,357762.266366
...,...,...,...,...,...,...,...
1844,136-75.tif,136,75,450027.239107,462762.374402,452527.425987,460262.187522
1845,136-76.tif,136,76,450027.433999,465262.565558,452527.630790,462762.368769
1846,136-77.tif,136,77,450027.433999,467762.762347,452527.630790,465262.565558
1847,136-78.tif,136,78,450026.847407,470262.978040,452527.065182,467762.760265


In [24]:
x[x['y'] == 108]

,file,y,x,ulx,uly,lrx,lry
330,108-44.tif,108,44,380027.258854,385262.853927,382527.400986,382762.711794
331,108-45.tif,108,45,380027.258854,387762.996060,382527.400986,385262.853927
332,108-46.tif,108,46,380027.258854,390262.138535,382527.400986,387762.996060
333,108-47.tif,108,47,380027.258854,392762.280668,382527.400986,390262.138535
334,108-48.tif,108,48,380027.258854,395262.422801,382527.400986,392762.280668
335,108-49.tif,108,49,380027.343179,397762.535718,382527.478481,395262.400417
336,108-50.tif,108,50,380027.350034,400262.669758,382527.484757,397762.535034
337,108-51.tif,108,51,380027.346180,402762.805899,382527.481191,400262.670889
338,108-52.tif,108,52,380027.339257,405262.942336,382527.474940,402762.806656
339,108-53.tif,108,53,380027.339257,407762.078361,382527.474940,405262.942336


In [26]:
x['y'].unique()

array([105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 135, 136])

In [43]:
## start

In [28]:
pos = 108
x1 = x[x['y'] == pos]
x1

,file,y,x,ulx,uly,lrx,lry
330,108-44.tif,108,44,380027.258854,385262.853927,382527.400986,382762.711794
331,108-45.tif,108,45,380027.258854,387762.996060,382527.400986,385262.853927
332,108-46.tif,108,46,380027.258854,390262.138535,382527.400986,387762.996060
333,108-47.tif,108,47,380027.258854,392762.280668,382527.400986,390262.138535
334,108-48.tif,108,48,380027.258854,395262.422801,382527.400986,392762.280668
335,108-49.tif,108,49,380027.343179,397762.535718,382527.478481,395262.400417
336,108-50.tif,108,50,380027.350034,400262.669758,382527.484757,397762.535034
337,108-51.tif,108,51,380027.346180,402762.805899,382527.481191,400262.670889
338,108-52.tif,108,52,380027.339257,405262.942336,382527.474940,402762.806656
339,108-53.tif,108,53,380027.339257,407762.078361,382527.474940,405262.942336


In [29]:
x2 = x1[(t_lry <= x1['uly']) & (t_uly >= x1['lry'])]
x2

,file,y,x,ulx,uly,lrx,lry
330,108-44.tif,108,44,380027.258854,385262.853927,382527.400986,382762.711794
331,108-45.tif,108,45,380027.258854,387762.996060,382527.400986,385262.853927
332,108-46.tif,108,46,380027.258854,390262.138535,382527.400986,387762.996060
333,108-47.tif,108,47,380027.258854,392762.280668,382527.400986,390262.138535
334,108-48.tif,108,48,380027.258854,395262.422801,382527.400986,392762.280668
335,108-49.tif,108,49,380027.343179,397762.535718,382527.478481,395262.400417
336,108-50.tif,108,50,380027.350034,400262.669758,382527.484757,397762.535034
337,108-51.tif,108,51,380027.346180,402762.805899,382527.481191,400262.670889
338,108-52.tif,108,52,380027.339257,405262.942336,382527.474940,402762.806656
339,108-53.tif,108,53,380027.339257,407762.078361,382527.474940,405262.942336


In [31]:
x2.iloc[:,0]

330    108-44.tif
331    108-45.tif
332    108-46.tif
333    108-47.tif
334    108-48.tif
335    108-49.tif
336    108-50.tif
337    108-51.tif
338    108-52.tif
339    108-53.tif
340    108-54.tif
341    108-55.tif
342    108-56.tif
343    108-57.tif
Name: file, dtype: object

In [33]:
x2.iloc[:,0].to_list()

['108-44.tif',
 '108-45.tif',
 '108-46.tif',
 '108-47.tif',
 '108-48.tif',
 '108-49.tif',
 '108-50.tif',
 '108-51.tif',
 '108-52.tif',
 '108-53.tif',
 '108-54.tif',
 '108-55.tif',
 '108-56.tif',
 '108-57.tif']

In [36]:
x2['file'].to_list()

['108-44.tif',
 '108-45.tif',
 '108-46.tif',
 '108-47.tif',
 '108-48.tif',
 '108-49.tif',
 '108-50.tif',
 '108-51.tif',
 '108-52.tif',
 '108-53.tif',
 '108-54.tif',
 '108-55.tif',
 '108-56.tif',
 '108-57.tif']

In [6]:
import os
print(os.path.exists("/media/philipp/5e9929a4-cb93-48b8-8648-6c83a93e83ed/gis/dtm"))

True


In [56]:
## mosaik two images

In [57]:
# mosaic two images together
import os, sys, gdal
from gdalconst import *
import numpy as np
import math

In [58]:
cd = '/media/philipp/5e9929a4-cb93-48b8-8648-6c83a93e83ed/gis/dtm/'
for file_name in x2['file']:
    print(cd + file_name)

/media/philipp/5e9929a4-cb93-48b8-8648-6c83a93e83ed/gis/dtm/108-43.tif
/media/philipp/5e9929a4-cb93-48b8-8648-6c83a93e83ed/gis/dtm/108-44.tif
/media/philipp/5e9929a4-cb93-48b8-8648-6c83a93e83ed/gis/dtm/108-45.tif
/media/philipp/5e9929a4-cb93-48b8-8648-6c83a93e83ed/gis/dtm/108-46.tif
/media/philipp/5e9929a4-cb93-48b8-8648-6c83a93e83ed/gis/dtm/108-47.tif
/media/philipp/5e9929a4-cb93-48b8-8648-6c83a93e83ed/gis/dtm/108-48.tif
/media/philipp/5e9929a4-cb93-48b8-8648-6c83a93e83ed/gis/dtm/108-49.tif
/media/philipp/5e9929a4-cb93-48b8-8648-6c83a93e83ed/gis/dtm/108-50.tif
/media/philipp/5e9929a4-cb93-48b8-8648-6c83a93e83ed/gis/dtm/108-51.tif
/media/philipp/5e9929a4-cb93-48b8-8648-6c83a93e83ed/gis/dtm/108-52.tif
/media/philipp/5e9929a4-cb93-48b8-8648-6c83a93e83ed/gis/dtm/108-53.tif
/media/philipp/5e9929a4-cb93-48b8-8648-6c83a93e83ed/gis/dtm/108-54.tif
/media/philipp/5e9929a4-cb93-48b8-8648-6c83a93e83ed/gis/dtm/108-55.tif
/media/philipp/5e9929a4-cb93-48b8-8648-6c83a93e83ed/gis/dtm/108-56.tif
/media

In [59]:
def mosaic(path1, path2, pathout):
    
    # register all of the GDAL drivers
    gdal.AllRegister()

    # read in doq1 and get info about it
    ds1 = gdal.Open(path1)
    band1 = ds1.GetRasterBand(1)
    rows1 = ds1.RasterYSize
    cols1 = ds1.RasterXSize

    # get the corner coordinates for doq1
    transform1 = ds1.GetGeoTransform()
    minX1 = transform1[0]
    maxY1 = transform1[3]
    pixelWidth1 = transform1[1]
    pixelHeight1 = transform1[5]
    maxX1 = minX1 + (cols1 * pixelWidth1)
    minY1 = maxY1 + (rows1 * pixelHeight1)

    # read in doq2 and get info about it
    ds2 = gdal.Open(path2)
    band2 = ds2.GetRasterBand(1)
    rows2 = ds2.RasterYSize
    cols2 = ds2.RasterXSize

    # get the corner coordinates for doq2
    transform2 = ds2.GetGeoTransform()
    minX2 = transform2[0]
    maxY2 = transform2[3]
    pixelWidth2 = transform2[1]
    pixelHeight2 = transform2[5]
    maxX2 = minX2 + (cols2 * pixelWidth2)
    minY2 = maxY2 + (rows2 * pixelHeight2)

    # get the corner coordinates for the output
    minX = min(minX1, minX2)
    maxX = max(maxX1, maxX2)
    minY = min(minY1, minY2)
    maxY = max(maxY1, maxY2)

    # get the number of rows and columns for the output
    cols = int(math.ceil((maxX - minX) / pixelWidth1))
    rows = int(math.ceil((maxY - minY) / abs(pixelHeight1)))

    # compute the origin (upper left) offset for doq1
    xOffset1 = int((minX1 - minX) / pixelWidth1)
    yOffset1 = int((maxY1 - maxY) / pixelHeight1)

    # compute the origin (upper left) offset for doq2
    xOffset2 = int((minX2 - minX) / pixelWidth1)
    yOffset2 = int((maxY2 - maxY) / pixelHeight1)

    # create the output image
    driver = ds1.GetDriver()
    dsOut = driver.Create(pathout, cols, rows, 1, band1.DataType)
    bandOut = dsOut.GetRasterBand(1)

    # read in doq1 and write it to the output
    data1 = band1.ReadAsArray(0, 0, cols1, rows1)
    bandOut.WriteArray(data1, xOffset1, yOffset1)

    # read in doq2 and write it to the output
    data2 = band2.ReadAsArray(0, 0, cols2, rows2)
    bandOut.WriteArray(data2, xOffset2, yOffset2)

    # compute statistics for the output
    bandOut.FlushCache()
    stats = bandOut.GetStatistics(0, 1)

    # set the geotransform and projection on the output
    geotransform = [minX, pixelWidth1, 0, maxY, 0, pixelHeight1]
    dsOut.SetGeoTransform(geotransform)
    dsOut.SetProjection(ds1.GetProjection())

    # build pyramids for the output
    #gdal.SetConfigOption('HFA_USE_RRD', 'YES')
    #dsOut.BuildOverviews(overviewlist=[2,4,8,16])



In [60]:
hd_path = '/media/philipp/5e9929a4-cb93-48b8-8648-6c83a93e83ed/gis/dtm/'

path1 = hd_path + x2.iloc[0,0]

for i in range(1, x2.shape[0]):
    #print(hd_path + df_0.iloc[i,0])

    path2 = hd_path + x2.iloc[i,0]
    print(path1, path2)
    
    pathout = '/media/philipp/5e9929a4-cb93-48b8-8648-6c83a93e83ed/gis/dtm_/mosaic_' + str(pos) + '.tif'
    mosaic(path1, path2, pathout)
    
    path1 = '/media/philipp/5e9929a4-cb93-48b8-8648-6c83a93e83ed/gis/dtm_/mosaic_' + str(pos) + '.tif'
    
print('end')

/media/philipp/5e9929a4-cb93-48b8-8648-6c83a93e83ed/gis/dtm/108-43.tif /media/philipp/5e9929a4-cb93-48b8-8648-6c83a93e83ed/gis/dtm/108-44.tif
/media/philipp/5e9929a4-cb93-48b8-8648-6c83a93e83ed/gis/dtm_/mosaic_108.tif /media/philipp/5e9929a4-cb93-48b8-8648-6c83a93e83ed/gis/dtm/108-45.tif
/media/philipp/5e9929a4-cb93-48b8-8648-6c83a93e83ed/gis/dtm_/mosaic_108.tif /media/philipp/5e9929a4-cb93-48b8-8648-6c83a93e83ed/gis/dtm/108-46.tif
/media/philipp/5e9929a4-cb93-48b8-8648-6c83a93e83ed/gis/dtm_/mosaic_108.tif /media/philipp/5e9929a4-cb93-48b8-8648-6c83a93e83ed/gis/dtm/108-47.tif
/media/philipp/5e9929a4-cb93-48b8-8648-6c83a93e83ed/gis/dtm_/mosaic_108.tif /media/philipp/5e9929a4-cb93-48b8-8648-6c83a93e83ed/gis/dtm/108-48.tif
/media/philipp/5e9929a4-cb93-48b8-8648-6c83a93e83ed/gis/dtm_/mosaic_108.tif /media/philipp/5e9929a4-cb93-48b8-8648-6c83a93e83ed/gis/dtm/108-49.tif
/media/philipp/5e9929a4-cb93-48b8-8648-6c83a93e83ed/gis/dtm_/mosaic_108.tif /media/philipp/5e9929a4-cb93-48b8-8648-6c83a93e

In [61]:
cd = '/media/philipp/5e9929a4-cb93-48b8-8648-6c83a93e83ed/gis/dtm_/'
directory = os.fsencode(cd)

paths = []

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".tif"): 
        #print(os.path.join(directory, filename))
        paths.append(cd + filename)
        
paths = np.sort(paths)

pathout = '/home/philipp/Data/edin/diss/dtm/out.tif'

print(paths)
print(pathout)

['/media/philipp/5e9929a4-cb93-48b8-8648-6c83a93e83ed/gis/dtm_/mosaic_106.tif'
 '/media/philipp/5e9929a4-cb93-48b8-8648-6c83a93e83ed/gis/dtm_/mosaic_107.tif'
 '/media/philipp/5e9929a4-cb93-48b8-8648-6c83a93e83ed/gis/dtm_/mosaic_108.tif']
/home/philipp/Data/edin/diss/dtm/out.tif


In [62]:
hd_path = '/media/philipp/5e9929a4-cb93-48b8-8648-6c83a93e83ed/gis/dtm/'

path1 = paths[0]

for i in range(1, len(paths)):
    #print(hd_path + df_0.iloc[i,0])

    path2 = paths[i]
    print(path1, path2)
    
    pathout = '/media/philipp/5e9929a4-cb93-48b8-8648-6c83a93e83ed/gis/dtm_/out.tif'
    mosaic(path1, path2, pathout)
    
    path1 = '/media/philipp/5e9929a4-cb93-48b8-8648-6c83a93e83ed/gis/dtm_/out.tif'
    
print('end')

/media/philipp/5e9929a4-cb93-48b8-8648-6c83a93e83ed/gis/dtm_/mosaic_106.tif /media/philipp/5e9929a4-cb93-48b8-8648-6c83a93e83ed/gis/dtm_/mosaic_107.tif
/media/philipp/5e9929a4-cb93-48b8-8648-6c83a93e83ed/gis/dtm_/out.tif /media/philipp/5e9929a4-cb93-48b8-8648-6c83a93e83ed/gis/dtm_/mosaic_108.tif
end


In [ ]:
## end

In [2]:
cd = '/home/philipp/Data/edin/diss/dtm/'
directory = os.fsencode(cd)

paths = []

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".tif"): 
        #print(os.path.join(directory, filename))
        paths.append(cd + filename)
        
paths = np.sort(paths)

pathout = '/home/philipp/Data/edin/diss/dtm/out.tif'

print(paths)
print(pathout)

['/home/philipp/Data/edin/diss/dtm/16-25.tif'
 '/home/philipp/Data/edin/diss/dtm/16-26.tif'
 '/home/philipp/Data/edin/diss/dtm/16-27.tif'
 '/home/philipp/Data/edin/diss/dtm/16-28.tif'
 '/home/philipp/Data/edin/diss/dtm/17-23.tif'
 '/home/philipp/Data/edin/diss/dtm/17-24.tif'
 '/home/philipp/Data/edin/diss/dtm/17-25.tif'
 '/home/philipp/Data/edin/diss/dtm/17-26.tif'
 '/home/philipp/Data/edin/diss/dtm/17-27.tif'
 '/home/philipp/Data/edin/diss/dtm/17-28.tif'
 '/home/philipp/Data/edin/diss/dtm/out.tif']
/home/philipp/Data/edin/diss/dtm/out.tif


In [3]:
for path in paths[:2]:
    print(path)

/home/philipp/Data/edin/diss/dtm/16-25.tif
/home/philipp/Data/edin/diss/dtm/16-26.tif


In [4]:
# register all of the GDAL drivers
gdal.AllRegister()

data = []
minmax = []

# read in doq1 and get info about it
for path in paths[:3]:
    ds1 = gdal.Open(path)
    band1 = ds1.GetRasterBand(1)
    rows1 = ds1.RasterYSize
    cols1 = ds1.RasterXSize

    # get the corner coordinates for doq1
    transform1 = ds1.GetGeoTransform()
    minX1 = transform1[0]
    maxY1 = transform1[3]
    pixelWidth1 = transform1[1]
    pixelHeight1 = transform1[5]
    maxX1 = minX1 + (cols1 * pixelWidth1)
    minY1 = maxY1 + (rows1 * pixelHeight1)

    data.append([band1, rows1, cols1])
    minmax.append([minX1, minY1, maxX1, maxY1, pixelWidth1, pixelHeight1, 0, 0])

# get the corner coordinates for the output
minmax = np.array(minmax)
minX = np.min(minmax[:,0])
minY = np.min(minmax[:,1])
maxX = np.max(minmax[:,2])
maxY = np.max(minmax[:,3])
print('minmax: ', minX, minY, maxX, maxY)

pixelWidth = minmax[0,4]
pixelHeight = minmax[0,5]
print('width, height: ', pixelWidth, pixelHeight)

# get the number of rows and columns for the output
cols = int((maxX - minX) / pixelWidth)
rows = int((maxY - minY) / abs(pixelHeight))
print('cols, rows: ', cols, rows)

for row in minmax:
    # compute the origin (upper left) offset for doq1
    row[6] = int((row[0] - minX) / pixelWidth)
    row[7] = int((row[3] - maxY) / pixelHeight)

minmax:  151456.4309958055 335311.2015465449 152527.69353754784 342762.29708616046
width, height:  0.9997129237 -0.9997129222
cols, rows:  1071 7453


In [5]:
print(pathout, cols, rows, 1, band1.DataType)
driver = ds1.GetDriver()
print(driver)
dsOut = driver.Create(pathout, cols, rows, 1, band1.DataType)
print(dsOut)
bandOut = dsOut.GetRasterBand(1)

/home/philipp/Data/edin/diss/dtm/out.tif 1071 7453 1 7
<osgeo.gdal.Driver; proxy of <Swig Object of type 'GDALDriverShadow *' at 0x7f0602527cf0> >
<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x7f06025272d0> >


In [8]:
data[0][0]

<osgeo.gdal.Band; proxy of <Swig Object of type 'GDALRasterBandShadow *' at 0x7f06537c2d20> >

In [ ]:
d = data[0][0].ReadAsArray(0, 0, data[0][2], data[0][1])

In [ ]:
for i in range(len(data)):
    # read in doq1 and write it to the output
    bandOut.WriteArray(data[i][0].ReadAsArray(0, 0, data[i][2], data[i][1]), minmax[i,6], minmax[i,7])

In [7]:
bandOut.FlushCache()
stats = bandOut.GetStatistics(0, 1)

In [8]:
stats

[-9999.0, 0.0, -5048.960024228443, 4999.255340148963]

In [5]:
#minmax:  151456.4309958055 335311.2015465449 152527.54183239024 340262.90218317154
#cols, rows:  1071 4953
#/home/philipp/Data/edin/diss/dtm/out.tif 1071 4953 1 7
#<osgeo.gdal.Driver; proxy of <Swig Object of type 'GDALDriverShadow *' at 0x7fe0a07cd630> >
#<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x7fe1005ef0f0> >


In [6]:
minmax

array([[ 1.51456849e+05,  3.35311202e+05,  1.52527542e+05,
         3.37762498e+05,  9.99712924e-01, -9.99712922e-01,
         0.00000000e+00,  2.50100000e+03],
       [ 1.51456431e+05,  3.37762631e+05,  1.52527119e+05,
         3.40262902e+05,  9.99708762e-01, -9.99708762e-01,
         0.00000000e+00,  0.00000000e+00]])

In [7]:
def mosaic(paths, pathout):
    '''
    paths = list of input paths
    pathout = output path
    '''
    
    # register all of the GDAL drivers
    gdal.AllRegister()
    
    data = []
    minmax = []
    
    # read in doq1 and get info about it
    for path in paths:
        ds1 = gdal.Open(path)
        band1 = ds1.GetRasterBand(1)
        rows1 = ds1.RasterYSize
        cols1 = ds1.RasterXSize

        # get the corner coordinates for doq1
        transform1 = ds1.GetGeoTransform()
        minX1 = transform1[0]
        maxY1 = transform1[3]
        pixelWidth1 = transform1[1]
        pixelHeight1 = transform1[5]
        maxX1 = minX1 + (cols1 * pixelWidth1)
        minY1 = maxY1 + (rows1 * pixelHeight1)
        
        data.append([band1, rows1, cols1])
        minmax.append([minX1, minY1, maxX1, maxY1, pixelWidth1, pixelHeight1, 0, 0])

    # get the corner coordinates for the output
    minmax = np.array(minmax)
    minX = np.min(minmax[:,0])
    minY = np.min(minmax[:,1])
    maxX = np.max(minmax[:,2])
    maxY = np.max(minmax[:,3])
    print('minmax: ', minX, minY, maxX, maxY)
    
    pixelWidth = minmax[0,4]
    pixelHeight = minmax[0,5]
    print('width, height: ', pixelWidth, pixelHeight)

    # get the number of rows and columns for the output
    cols = int((maxX - minX) / pixelWidth)
    rows = int((maxY - minY) / abs(pixelHeight))
    print('cols, rows: ', cols, rows)
    
    for row in minmax:
        # compute the origin (upper left) offset for doq1
        row[6] = int((row[0] - minX) / pixelWidth)
        row[7] = int((row[3] - maxY) / pixelHeight)

    print(pathout, cols, rows, 1, band1.DataType)
    driver = ds1.GetDriver()
    print(driver)
    dsOut = driver.Create(pathout, cols, rows, 1, band1.DataType)
    print(dsOut)
    bandOut = dsOut.GetRasterBand(1)
    
    for i in range(len(data)):
        # read in doq1 and write it to the output
        data1 = data[i][0].ReadAsArray(0, 0, data[i][2], data[i][1])
        bandOut.WriteArray(data1, minmax[i,6], minmax[i,7])

        # read in doq2 and write it to the output
        #data2 = band2.ReadAsArray(0, 0, cols2, rows2)
        #bandOut.WriteArray(data2, xOffset2, yOffset2)

    # compute statistics for the output
    bandOut.FlushCache()
    stats = bandOut.GetStatistics(0, 1)

    # set the geotransform and projection on the output
    geotransform = [minX, pixelWidth1, 0, maxY, 0, pixelHeight1]
    dsOut.SetGeoTransform(geotransform)
    dsOut.SetProjection(ds1.GetProjection())

    # build pyramids for the output
    #gdal.SetConfigOption('HFA_USE_RRD', 'YES')
    #dsOut.BuildOverviews(overviewlist=[2,4,8,16])

In [8]:
paths[:2]

array(['/home/philipp/Data/edin/diss/dtm/16-25.tif',
       '/home/philipp/Data/edin/diss/dtm/16-26.tif'], dtype='<U42')

In [9]:
#minmax:  151456.4309958055 337762.63056915946 152527.69353754784 342762.29708616046
#cols, rows:  1071 5001
#/home/philipp/Data/edin/diss/dtm/out.tif 1071 5001 1 7
#<osgeo.gdal.Driver; proxy of <Swig Object of type 'GDALDriverShadow *' at 0x7f40b28272d0> >
#<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x7f40b2839ba0> >

In [ ]:
mosaic(paths[:2], pathout)

In [27]:
def mosaic(path1, path2, pathout):
    
    # register all of the GDAL drivers
    gdal.AllRegister()

    # read in doq1 and get info about it
    ds1 = gdal.Open(path1)
    band1 = ds1.GetRasterBand(1)
    rows1 = ds1.RasterYSize
    cols1 = ds1.RasterXSize

    # get the corner coordinates for doq1
    transform1 = ds1.GetGeoTransform()
    minX1 = transform1[0]
    maxY1 = transform1[3]
    pixelWidth1 = transform1[1]
    pixelHeight1 = transform1[5]
    maxX1 = minX1 + (cols1 * pixelWidth1)
    minY1 = maxY1 + (rows1 * pixelHeight1)

    # read in doq2 and get info about it
    ds2 = gdal.Open(path2)
    band2 = ds2.GetRasterBand(1)
    rows2 = ds2.RasterYSize
    cols2 = ds2.RasterXSize

    # get the corner coordinates for doq2
    transform2 = ds2.GetGeoTransform()
    minX2 = transform2[0]
    maxY2 = transform2[3]
    pixelWidth2 = transform2[1]
    pixelHeight2 = transform2[5]
    maxX2 = minX2 + (cols2 * pixelWidth2)
    minY2 = maxY2 + (rows2 * pixelHeight2)

    # get the corner coordinates for the output
    minX = min(minX1, minX2)
    maxX = max(maxX1, maxX2)
    minY = min(minY1, minY2)
    maxY = max(maxY1, maxY2)
    print('minmax: ', minX, minY, maxX, maxY)

    # get the number of rows and columns for the output
    cols = int((maxX - minX) / pixelWidth1)
    rows = int((maxY - minY) / abs(pixelHeight1))
    cols += 1
    print('cols, rows: ', cols, rows)

    # compute the origin (upper left) offset for doq1
    xOffset1 = int((minX1 - minX) / pixelWidth1)
    yOffset1 = int((maxY1 - maxY) / pixelHeight1)
    print('xOffset1, yOffset1: ', xOffset1, yOffset1)

    # compute the origin (upper left) offset for doq2
    xOffset2 = int((minX2 - minX) / pixelWidth1)
    yOffset2 = int((maxY2 - maxY) / pixelHeight1)
    
    # create the output image
    driver = ds1.GetDriver()
    dsOut = driver.Create(pathout, cols, rows, 1, band1.DataType)
    bandOut = dsOut.GetRasterBand(1)

    # read in doq1 and write it to the output
    data1 = band1.ReadAsArray(0, 0, cols1, rows1)
    bandOut.WriteArray(data1, xOffset1, yOffset1)

    # read in doq2 and write it to the output
    data2 = band2.ReadAsArray(0, 0, cols2, rows2)
    bandOut.WriteArray(data2, xOffset2, yOffset2)

    # compute statistics for the output
    bandOut.FlushCache()
    stats = bandOut.GetStatistics(0, 1)

    # set the geotransform and projection on the output
    geotransform = [minX, pixelWidth1, 0, maxY, 0, pixelHeight1]
    dsOut.SetGeoTransform(geotransform)
    dsOut.SetProjection(ds1.GetProjection())

    # build pyramids for the output
    #gdal.SetConfigOption('HFA_USE_RRD', 'YES')
    #dsOut.BuildOverviews(overviewlist=[2,4,8,16])

In [29]:
path1 = '/media/philipp/5e9929a4-cb93-48b8-8648-6c83a93e83ed/gis/dtm/105-43.tif'
path2 = '/media/philipp/5e9929a4-cb93-48b8-8648-6c83a93e83ed/gis/dtm/105-44.tif'
pathout = '/media/philipp/5e9929a4-cb93-48b8-8648-6c83a93e83ed/gis/dtm_/out.tif'

mosaic(path1, path2, pathout)

minmax:  372526.85594610183 380262.65569293976 375027.01510271337 385262.91983602726
cols, rows:  2502 5001
xOffset1, yOffset1:  0 2500


In [28]:
path1 = '/media/philipp/5e9929a4-cb93-48b8-8648-6c83a93e83ed/gis/dtm/106-43.tif'
path2 = '/media/philipp/5e9929a4-cb93-48b8-8648-6c83a93e83ed/gis/dtm/106-44.tif'
pathout = '/media/philipp/5e9929a4-cb93-48b8-8648-6c83a93e83ed/gis/dtm_/out2.tif'

mosaic(path1, path2, pathout)

minmax:  375026.9745905141 380262.5696613421 377527.1167225041 385262.85392682266
cols, rows:  2501 5001
xOffset1, yOffset1:  0 2500


In [5]:
path1 = '/home/philipp/Data/edin/diss/dtm/out.tif'
path2 = paths[3]
pathout = '/home/philipp/Data/edin/diss/dtm/out2.tif'

mosaic(path1, path2, pathout)

minmax:  151456.4309958055 335311.32407951495 152527.69353754784 342901.2560469248
cols, rows:  1071 7592
/home/philipp/Data/edin/diss/dtm/out2.tif 1071 7592 1 7
<osgeo.gdal.Driver; proxy of <Swig Object of type 'GDALDriverShadow *' at 0x7f91fb31e2a0> >
<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x7f91fb31ec30> >
[-9999.0, 0.0, -6673.39930466418, 4710.951216882356]


In [7]:
path1 = '/home/philipp/Data/edin/diss/dtm/out2.tif'
path2 = paths[5]
pathout = '/home/philipp/Data/edin/diss/dtm/out2.tif'

mosaic(path1, path2, pathout)

minmax:  151456.4309958055 332762.3537871272 155026.96443154037 342901.2560469248
cols, rows:  3571 10141
/home/philipp/Data/edin/diss/dtm/out2.tif 3571 10141 1 7
<osgeo.gdal.Driver; proxy of <Swig Object of type 'GDALDriverShadow *' at 0x7f91fb342b40> >
<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x7f92585dec90> >
[-9999.0, 0.0, -2306.332493554664, 4212.107433604467]
